# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [4]:
driver = webdriver.Chrome()

In [5]:
driver.get("https://www.tdlr.texas.gov/tools_search/")

In [6]:
text_input = driver.find_element_by_xpath('//*[@id="mcrdata"]')
text_input.send_keys('006179570C')

In [7]:
button = driver.find_element_by_xpath('//*[@id="submit3"]/b')
button.click()

In [8]:
company_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]')
company_name.text


'Name:   B.D. SMITH TOWING'

In [10]:
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]')
owner.text

'Owner/Officer:   BRANDT SMITH / OWNER'

In [11]:
phone_number = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]')
phone_number.text

'Phone:   8173330706'

In [12]:
license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font')
license_status.text

'Expired'

In [17]:
address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
address1 = address.split(' ')[19:26]
address1

['76179\n\nPhysical:\n13619',
 'BRETT',
 'JACKSON',
 'RD.\nFORT',
 'WORTH,',
 'TX.',
 '76179']

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [25]:
tow_trucks = []

tow_info = {}
tow_info["company_name"] = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
tow_info["owner"] = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
tow_info["phone"] = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
tow_info["license_status"] = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
tow_info["address"] = address.split(' ')[19:26]

tow_trucks.append(tow_info)
    
tow_trucks

[{'company_name': 'Name:   B.D. SMITH TOWING',
  'owner': 'Owner/Officer:   BRANDT SMITH / OWNER',
  'phone': 'Phone:   8173330706',
  'license_status': 'Expired',
  'address': ['76179\n\nPhysical:\n13619',
   'BRETT',
   'JACKSON',
   'RD.\nFORT',
   'WORTH,',
   'TX.',
   '76179']}]

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [29]:
import pandas as pd
df = pd.read_csv("trucks-subset.csv")
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [30]:
df = df.rename(columns={'TDLR Number': 'TDLR_Number'})
df

,TDLR_Number
0,006507931C
1,006179570C
2,006502097C


In [59]:
def get_links(row):
    for rows in df['TDLR_Number']:
        return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row
  

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [60]:
df['url'] = df.TDLR_Number.apply(get_links)

In [61]:
pd.set_option('display.max_colwidth', -1)
df

,TDLR_Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [62]:
def get_truck_info(row):
    driver.get("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR_Number'])
    
    company_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    license_status1 = "License status:" + license_status
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    address1 = address.split(' ')[19:26]
    
    print(company_name, owner, phone, license_status1, address1)

In [63]:
df.apply(get_truck_info, axis=1)

Name:   AUGUSTUS E SMITH Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER Phone:   9032276464 License status:Active ['N', 'MAIN', 'ST\nBONHAM,', 'TX.', '75418']
Name:   B.D. SMITH TOWING Owner/Officer:   BRANDT SMITH / OWNER Phone:   8173330706 License status:Expired ['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSON', 'RD.\nFORT', 'WORTH,', 'TX.', '76179']
Name:   BARRY MICHAEL SMITH Owner/Officer:   BARRY MICHAEL SMITH / OWNER Phone:   8066544404 License status:Active ['W', 'CEMETERY', 'RD\nCANYON,', 'TX.', '79015']


0    None
1    None
2    None
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [64]:
def get_truck_info(row):
    driver.get("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR_Number'])
    
    company_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    address1 = address.split(' ')[19:26]
    
    return pd.Series({
        "company_name": company_name,
        "owner": owner,
        "phone": phone,
        "license_status": license_status,
        "address": address1
    })


In [65]:
driver = webdriver.Chrome()

new_df = df.apply(get_truck_info, axis=1).join(df)
new_df.head()

,company_name,owner,phone,license_status,address,TDLR_Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Active,"[N, MAIN, ST\nBONHAM,, TX., 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Expired,"[76179\n\nPhysical:\n13619, BRETT, JACKSON, RD.\nFORT, WORTH,, TX., 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Active,"[W, CEMETERY, RD\nCANYON,, TX., 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [68]:
new_df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [69]:
new_df = pd.read_csv("tow-trucks-extended.csv")
new_df

,company_name,owner,phone,license_status,address,TDLR_Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Active,"['N', 'MAIN', 'ST\nBONHAM,', 'TX.', '75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Expired,"['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSON', 'RD.\nFORT', 'WORTH,', 'TX.', '76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Active,"['W', 'CEMETERY', 'RD\nCANYON,', 'TX.', '79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [76]:
df = pd.read_csv("tow-trucks.csv")
df = df.rename(columns={'TDLR Number': 'TDLR_Number'})


In [78]:
def get_links(row):
    for rows in df['TDLR_Number']:
        return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row
  
df['url'] = df.TDLR_Number.apply(get_links)

pd.set_option('display.max_colwidth', -1)

In [79]:
def get_truck_info(row):
    driver.get("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR_Number'])
    
    company_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    address1 = address.split(' ')[19:26]
    
    return pd.Series({
        "company_name": company_name,
        "owner": owner,
        "phone": phone,
        "license_status": license_status,
        "address": address1
    
    })

In [84]:
driver = webdriver.Chrome()

new_df = df.apply(get_truck_info, axis=1).join(df)
new_df.head()

,company_name,owner,phone,license_status,address,TDLR_Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Active,"[N, MAIN, ST\nBONHAM,, TX., 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Expired,"[76179\n\nPhysical:\n13619, BRETT, JACKSON, RD.\nFORT, WORTH,, TX., 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Active,"[W, CEMETERY, RD\nCANYON,, TX., 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Expired,"[ST\nVERNON,, TX., 76384]",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Expired,[],0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF


In [85]:
new_df.to_csv("tow-trucks-extended.csv", index=False)